In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv
/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv
/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/gender_submission.csv


# **Tiền xử lí dữ liệu**

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Đọc dữ liệu từ file CSV
train_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-hk-1-se-355-p-11-titanic-classification/test.csv')

# Xử lý giá trị thiếu trong dữ liệu huấn luyện
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())

# Tạo thêm đặc trưng mới trong dữ liệu huấn luyện
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)

# Chuyển đổi các biến phân loại thành số trong dữ liệu huấn luyện
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked'], drop_first=True)

# Loại bỏ các cột không cần thiết trong dữ liệu huấn luyện
train_data.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)

# Tách dữ liệu thành các đặc trưng và biến mục tiêu trong dữ liệu huấn luyện
X = train_data.drop(columns=['Survived', 'PassengerId'])
y = train_data['Survived']

# Chuẩn hóa các đặc trưng trong dữ liệu huấn luyện
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Xử lý giá trị thiếu trong dữ liệu test
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())
test_data['Embarked'] = test_data['Embarked'].fillna(test_data['Embarked'].mode()[0])
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

# Tạo thêm đặc trưng mới trong dữ liệu test
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)

# Chuyển đổi các biến phân loại thành số trong dữ liệu test
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked'], drop_first=True)

# Đảm bảo rằng dữ liệu test có cùng các cột với dữ liệu huấn luyện
missing_cols = set(train_data.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[train_data.columns.drop(['Survived'])]

# Chuẩn hóa dữ liệu test
X_test = scaler.transform(test_data.drop(columns=['PassengerId']))

# Hiển thị dữ liệu huấn luyện đã tiền xử lý
print("Dữ liệu huấn luyện đã tiền xử lý:")
print(train_data.head())

# Hiển thị dữ liệu test đã tiền xử lý
print("\nDữ liệu test đã tiền xử lý:")
print(test_data.head())

Dữ liệu huấn luyện đã tiền xử lý:
   PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  FamilySize  \
0            1         0       3  22.0      1      0   7.2500           2   
1            2         1       1  38.0      1      0  71.2833           2   
2            3         1       3  26.0      0      0   7.9250           1   
3            4         1       1  35.0      1      0  53.1000           2   
4            5         0       3  35.0      0      0   8.0500           1   

   IsAlone  Sex_male  Embarked_Q  Embarked_S  
0        0      True       False        True  
1        0     False       False       False  
2        1     False       False        True  
3        0     False       False        True  
4        1      True       False        True  

Dữ liệu test đã tiền xử lý:
   PassengerId  Pclass   Age  SibSp  Parch     Fare  FamilySize  IsAlone  \
0          892       3  34.5      0      0   7.8292           1        1   
1          893       3  47.0      1     

# **Train model**

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Hàm để huấn luyện và đánh giá mô hình
def train_and_evaluate_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    return model

# Huấn luyện và đánh giá các mô hình khác nhau

# Logistic Regression (tăng số lượng vòng lặp tối đa)
logreg = LogisticRegression(max_iter=1000)
logreg_model = train_and_evaluate_model(logreg, "Logistic Regression")

# Decision Tree Classifier
dtree = DecisionTreeClassifier()
dtree_model = train_and_evaluate_model(dtree, "Decision Tree Classifier")

# Random Forest Classifier
rf = RandomForestClassifier()
rf_model = train_and_evaluate_model(rf, "Random Forest Classifier")

# Support Vector Classifier
svc = SVC()
svc_model = train_and_evaluate_model(svc, "Support Vector Classifier")

# K-Nearest Neighbors Classifier
knn = KNeighborsClassifier()
knn_model = train_and_evaluate_model(knn, "K-Nearest Neighbors Classifier")

Logistic Regression Accuracy: 0.7988826815642458
Decision Tree Classifier Accuracy: 0.7877094972067039
Random Forest Classifier Accuracy: 0.8324022346368715
Support Vector Classifier Accuracy: 0.8156424581005587
K-Nearest Neighbors Classifier Accuracy: 0.8100558659217877


**1. Logistic Regression**

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logistic Regression (tăng số lượng vòng lặp tối đa)
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Logistic Regression Accuracy: {accuracy}")


Logistic Regression Accuracy: 0.7988826815642458


**2. K-Nearest Neighbors (KNN)**

In [34]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# K-Nearest Neighbors Classifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"K-Nearest Neighbors Classifier Accuracy: {accuracy}")


K-Nearest Neighbors Classifier Accuracy: 0.8100558659217877


**3. Decision Tree Classifier (CART)**

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Decision Tree Classifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Decision Tree Classifier Accuracy: {accuracy}")


Decision Tree Classifier Accuracy: 0.7821229050279329


**4. Random Forest**

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Random Forest Classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Random Forest Classifier Accuracy: {accuracy}")


Random Forest Classifier Accuracy: 0.8324022346368715


**5. Support Vector Classifier (SVC)**

In [35]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Support Vector Classifier
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Support Vector Classifier Accuracy: {accuracy}")


Support Vector Classifier Accuracy: 0.8156424581005587


# **Submit**

In [37]:
# Hàm để tạo file submission từ mô hình đã huấn luyện
def create_submission_file(model, model_name):
    predictions = model.predict(X_test)
    submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': predictions})
    submission.to_csv(f'submission_{model_name}.csv', index=False)
    print(f"Submission file for {model_name} created successfully.")

# Tạo file submission cho từng mô hình

create_submission_file(logreg_model, "Logistic_Regression")
create_submission_file(dtree_model, "Decision_Tree_Classifier")
create_submission_file(rf_model, "Random_Forest_Classifier")
create_submission_file(svc_model, "Support_Vector_Classifier")
create_submission_file(knn_model, "K_Nearest_Neighbors_Classifier")

Submission file for Logistic_Regression created successfully.
Submission file for Decision_Tree_Classifier created successfully.
Submission file for Random_Forest_Classifier created successfully.
Submission file for Support_Vector_Classifier created successfully.
Submission file for K_Nearest_Neighbors_Classifier created successfully.
